In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Veo 2 Editing

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb">
      <img src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fvision%2Fgetting-started%2Fveo2_editing.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/vision/getting-started/veo2_editing.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/veo2_editing.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
|Author(s) | [Katie Nguyen](https://github.com/katiemn) |

## Overview

### Veo 2

Veo 2 on Vertex AI brings Google's video generation capabilities to application developers. It's capable of creating videos with astonishing detail that simulate real-world physics across a wide range of visual styles.

In this tutorial, you will learn how to use the Google Gen AI SDK for Python to interact with Veo 2 Editing to:
- Remove content
- Insert objects


## Get started

### Install Google Gen AI SDK for Python

In [ ]:
%pip install --upgrade --quiet google-genai

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import libraries

In [ ]:
import time

from IPython.display import Image, Video, display
from google import genai
from google.genai import types

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Define helper functions

In [ ]:
def show_video(video):
    if isinstance(video, str):
        file_name = video.split("/")[-1]
        !gsutil cp {video} {file_name}
        display(Video(file_name, embed=True, width=600, height=400))
    else:
        with open("sample.mp4", "wb") as out_file:
            out_file.write(video)
        display(Video("sample.mp4", embed=True, width=600, height=400))

### Load the video model

In [ ]:
video_model = "veo-2.0-generate-preview"

### Load the starting video

Throughout this notebook, you'll edit the following video stored in Cloud Storage. Run the following cell to set the `video_gcs` variable you'll use in each request.

In [ ]:
video_gcs = "gs://cloud-samples-data/generative-ai/video/truck.mp4"
show_video(video_gcs)

## Remove content with inpainting

With inpainting removal, you can provide a mask to remove objects from a video. This can be done through static or dynamic inpainting.

### Static inpainting

Static inpainting applies a mask to each frame of the video, allowing you to replace the masked area with new content.

First, you'll download and display the mask image to be used in the request.


In [ ]:
# Download the mask
!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/truck-inpainting-remove-static-mask.png

# Display the mask
mask_image = "truck-inpainting-remove-static-mask.png"
display(Image(mask_image, height=400))

In order to edit the video with this mask, specify the following:

- **Prompt:** The new object you'd like to replace the masked content in each frame.
- **Prompt enhancement:** The model offers the option to enhance your provided prompt. To utilize this feature, set `enhance_prompt` to True. A new, detailed prompt will be created from your original one to help generate higher quality videos that better adhere to your prompt's intent.

**Safety:** All Veo videos include [SynthID](https://deepmind.google/technologies/synthid/).

In [ ]:
prompt = "a mountain landscape"  # @param {type: 'string'}

operation = client.models.generate_videos(
    model=video_model,
    source=types.GenerateVideosSource(
        prompt=prompt, video=types.Video(uri=video_gcs, mime_type="video/mp4")
    ),
    config=types.GenerateVideosConfig(
        mask=types.VideoGenerationMask(
            image=types.Image.from_file(location=mask_image),
            mask_mode=types.VideoGenerationMaskMode.REMOVE_STATIC,
        ),
        enhance_prompt=True,
    ),
)

while not operation.done:
    time.sleep(10)
    operation = client.operations.get(operation)
    print(operation)

if operation.response:
    show_video(operation.result.generated_videos[0].video.video_bytes)

### Dynamic inpainting
Dynamic inpainting uses a mask to select and remove an object from the first frame of a video. Only one object can be removed at a time.

This example uses a video and mask image downloaded from Cloud Storage.

In [ ]:
# Download the mask
!wget https://storage.googleapis.com/cloud-samples-data/generative-ai/image/truck-inpainting-dynamic-mask.png

# Display the mask
mask_image = "truck-inpainting-dynamic-mask.png"
display(Image(mask_image, height=400))

Since you're removing content from a video you don't need to specify a prompt.

In [ ]:
operation = client.models.generate_videos(
    model=video_model,
    source=types.GenerateVideosSource(
        video=types.Video(uri=video_gcs, mime_type="video/mp4")
    ),
    config=types.GenerateVideosConfig(
        mask=types.VideoGenerationMask(
            image=types.Image.from_file(location=mask_image),
            mask_mode=types.VideoGenerationMaskMode.REMOVE,
        ),
        enhance_prompt=True,
    ),
)

while not operation.done:
    time.sleep(10)
    operation = client.operations.get(operation)
    print(operation)

if operation.response:
    show_video(operation.result.generated_videos[0].video.video_bytes)

## Add objects with inpainting

With inpainting insertion, you can use a mask to add objects to a video.

This example uses the same video and a mask image from Cloud Storage.

In [ ]:
# fmt: off
mask_gcs = "gs://cloud-samples-data/generative-ai/image/truck-inpainting-dynamic-mask.png"
# fmt: on

# Display mask image
file_name = mask_gcs.split("/")[-1]
!gsutil cp {mask_gcs} {file_name}
display(Image(file_name, height=400))

**Output file location:** The generated video will be displayed below and saved to a Cloud Storage bucket. Specify the output path in the `output_gcs` variable.

In [13]:
output_gcs = "gs://[your-bucket-path]"  # @param {type: 'string'}

This request will be similar to the others, except you'll reference the mask image from Cloud Storage directly, rather than using a locally downloaded image. To add content in this request specify the following:
  - **Prompt:** The object you'd like added in the masked area.
  - **Mask GCS:** The Cloud Storage location of your image mask.
  - **Mask mime type:** Select either `image/png` or `image/jpeg`.

In [ ]:
prompt = "sheep"  # @param {type: 'string'}

operation = client.models.generate_videos(
    model=video_model,
    source=types.GenerateVideosSource(
        prompt=prompt, video=types.Video(uri=video_gcs, mime_type="video/mp4")
    ),
    config=types.GenerateVideosConfig(
        mask=types.VideoGenerationMask(
            image=types.Image(
                gcs_uri=mask_gcs,
                mime_type="image/png",
            ),
            mask_mode=types.VideoGenerationMaskMode.INSERT,
        ),
        output_gcs_uri=output_gcs,
        enhance_prompt=True,
    ),
)

while not operation.done:
    time.sleep(10)
    operation = client.operations.get(operation)
    print(operation)

if operation.response:
    show_video(operation.result.generated_videos[0].video.uri)